# Add SHN Info to Transit Routes in the middle of the Pipeline

In [8]:
import datetime

import geopandas as gpd
import google.auth
import numpy as np
import pandas as pd
import yaml
from calitp_data_analysis import geography_utils, utils
from calitp_data_analysis.geography_utils import WGS84
from segment_speed_utils import helpers
from shared_utils import (
    catalog_utils,
    portfolio_utils,
    publish_utils,
    rt_dates,
    rt_utils,
    schedule_rt_utils,
    shared_data
)
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS, SCHED_GCS, SEGMENT_GCS

credentials, project = google.auth.default()

import gcsfs

fs = gcsfs.GCSFileSystem()

In [9]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [10]:
analysis_date_list = rt_dates.y2025_dates

In [11]:
analysis_date_list[0]

'2025-01-15'

In [12]:
date = analysis_date_list[0]

## Move the bulk of the SHN work from `open_data/create_routes_data` to `_shared_utils/shared_data`

In [23]:
shn = shared_data.make_clean_state_highway_network()

HTTPError: HTTP Error 400: Bad Request

In [14]:
SHN_FILE = catalog_utils.get_catalog(
        "shared_data_catalog"
    ).state_highway_network.urlpath

shn = gpd.read_parquet(
        SHN_FILE,
        storage_options={"token": credentials.token},
    ).to_crs(geography_utils.CA_NAD83Albers_ft)

In [15]:
shn.columns

Index(['Route', 'County', 'District', 'RouteType', 'Direction', 'geometry'], dtype='object')

In [16]:
shn.drop(columns = ["geometry"]).sample(5)

,Route,County,District,RouteType,Direction
480,89,SIS,2,State,NB
63,5,SHA,2,Interstate,SB
1027,605,LA,7,Interstate,SB
1006,395,SBD,8,US,NB
699,145,MAD,6,State,SB


In [17]:
GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/shared_data/"

In [18]:
def dissolve_shn_district() -> gpd.GeoDataFrame:
    """
    Dissolve State Highway Network so there will only be one row for each
    route name, route type, and Caltrans district. Find the length 
    of the highway and do some light cleaning.
    """
    # Read in the dataset and change the CRS to one to feet.
    SHN_FILE = catalog_utils.get_catalog(
        "shared_data_catalog"
    ).state_highway_network.urlpath

    shn = gpd.read_parquet(
        SHN_FILE,
        storage_options={"token": credentials.token},
    ).to_crs(geography_utils.CA_NAD83Albers_ft)

    # Dissolve by route which represents the the route's name and drop the other columns
    # because they are no longer relevant.
    shn_dissolved = (shn.dissolve(by=["Route", "District"]).reset_index())[[
        "Route", "District", "geometry"]]

    # Rename because I don't want any confusion between SHN route and
    # transit route.
    shn_dissolved = shn_dissolved.rename(columns={"Route": "shn_route"})
    shn_dissolved.columns = shn_dissolved.columns.str.lower()
    # Find the length of each highway.
    shn_dissolved = shn_dissolved.assign(
        highway_feet=shn_dissolved.geometry.length,
        shn_route=shn_dissolved.shn_route.astype(int).astype(str),
    )

    # Save this out so I don't have to dissolve it each time.
    shn_dissolved.to_parquet(
        f"{GCS_FILE_PATH}shn_dissolved_by_ct_district_route.parquet",
        filesystem=fs,
    )
    return shn_dissolved

In [19]:
dissolved_route = dissolve_shn_district()

In [20]:
def buffer_shn(buffer_amount: int, file_name: str) -> gpd.GeoDataFrame:
    """
    Add a buffer to the SHN before overlaying it with
    transit routes.
    """
    # GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/state_highway_network/"

    # Read in the dissolved SHN file
    shn_df = gpd.read_parquet(
        f"{GCS_FILE_PATH}{file_name}.parquet",
        storage_options={"token": credentials.token},
    )

    # Buffer the state highway.
    shn_df_buffered = shn_df.assign(
        geometry=shn_df.geometry.buffer(buffer_amount),
    )

    # Save it out so we won't have to buffer over again and
    # can just read it in.
    shn_df_buffered.to_parquet(
        f"{GCS_FILE_PATH}shn_buffered_{buffer_amount}_ft_{file_name}.parquet",
        filesystem=fs,
    )

    return shn_df_buffered

In [21]:
SHN_HWY_BUFFER_FEET = 50
PARALLEL_HWY_BUFFER_FEET = geography_utils.FEET_PER_MI * 0.5

In [22]:
buffered_shn = buffer_shn(SHN_HWY_BUFFER_FEET,"shn_dissolved_by_ct_district_route")

## Test function in `create_routes_data`

In [25]:
trips = helpers.import_scheduled_trips(
    date,
    columns=[
        "gtfs_dataset_key",
        "route_id",
        "route_type",
        "shape_id",
        "shape_array_key",
        "route_long_name",
        "route_short_name",
        "route_desc",
    ],
    get_pandas=True,
).dropna(subset="shape_array_key")

In [26]:
trips.sample()

,schedule_gtfs_dataset_key,route_id,route_type,shape_id,shape_array_key,route_long_name,route_short_name,route_desc
5694,f8c7d523e77341ee3c140b24a316f0fa,12411,3,p_531105,0149644df72c907249f37aa8534b3b0f,Route 70,None,None


In [27]:
shapes = helpers.import_scheduled_shapes(
    date, columns=["shape_array_key", "n_trips", "geometry"], get_pandas=True, crs=WGS84
).dropna(subset="shape_array_key")

In [28]:
shapes.sample().drop(columns=["geometry"])

,shape_array_key,n_trips
1182,1c4d8a9df1aea910b6ac64dabe12521d,3


In [29]:
df = (
    pd.merge(shapes, trips, on="shape_array_key", how="inner")
    .drop_duplicates(subset="shape_array_key")
    .drop(columns="shape_array_key")
)

In [30]:
df.shape

(7416, 9)

In [31]:
drop_cols = ["route_short_name", "route_long_name", "route_desc"]
route_shape_cols = ["schedule_gtfs_dataset_key", "route_id", "shape_id"]

In [32]:
def remove_erroneous_shapes(
    shapes_with_route_info: gpd.GeoDataFrame,
) -> gpd.GeoDataFrame:
    """
    Check if line is simple for Amtrak. If it is, keep.
    If it's not simple (line crosses itself), drop.

    In Jun 2023, some Amtrak shapes appeared to be funky,
    but in prior months, it's been ok.
    Checking for length is fairly time-consuming.
    """
    amtrak = "Amtrak Schedule"

    possible_error = shapes_with_route_info[shapes_with_route_info.name == amtrak]
    ok = shapes_with_route_info[shapes_with_route_info.name != amtrak]

    # Check if the line crosses itself
    ok_amtrak = (
        possible_error.assign(simple=possible_error.geometry.is_simple)
        .query("simple == True")
        .drop(columns="simple")
    )

    ok_shapes = pd.concat([ok, ok_amtrak], axis=0).reset_index(drop=True)

    return ok_shapes

### Didn't reach the step of `routes_assmebled2` because of all the different imports causing issues.

In [33]:
routes_assembled = (
    portfolio_utils.add_route_name(df)
    .drop(columns=drop_cols)
    .sort_values(route_shape_cols)
    .drop_duplicates(subset=route_shape_cols)
    .reset_index(drop=True)
)

In [34]:
routes_assembled.shape

(7416, 7)

In [35]:
routes_assembled.columns

Index(['n_trips', 'geometry', 'schedule_gtfs_dataset_key', 'route_id',
       'route_type', 'shape_id', 'route_name_used'],
      dtype='object')

### Add length to the transit routes.

In [36]:
routes_assembled = routes_assembled.assign(
    route_length_feet=routes_assembled.geometry.to_crs(
        geography_utils.CA_NAD83Albers_ft
    ).length
)

## Overlay the transit routes with the SHN 

In [37]:
def routes_shn_intersection(
    routes_gdf: gpd.GeoDataFrame, buffer_amount: int
) -> gpd.GeoDataFrame:
    """
    Overlay the most recent transit routes with a buffered version
    of the SHN
    """
    GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/shared_data/"

    # Read in buffered shn here or re buffer if we don't have it available.
    HWY_FILE = f"{GCS_FILE_PATH}shn_buffered_{buffer_amount}_ft_shn_dissolved_by_ct_district_route.parquet"

    if fs.exists(HWY_FILE):
        shn_routes_gdf = gpd.read_parquet(
            HWY_FILE, storage_options={"token": credentials.token}
        )
    else:
        shn_routes_gdf = shared_data.buffer_shn(buffer_amount)

    # Process the most recent transit route geographies and ensure the
    # CRS matches the SHN routes' GDF so the overlay doesn't go wonky.
    routes_gdf = routes_gdf.to_crs(shn_routes_gdf.crs)

    # Overlay transit routes with the SHN geographies.
    gdf = gpd.overlay(
        routes_gdf, shn_routes_gdf, how="intersection", keep_geom_type=True
    )

    # Calcuate the percent of the transit route that runs on a highway, round it up and
    # multiply it by 100. Drop the geometry because we want the original transit route
    # shapes.
    gdf = gdf.assign(
        pct_route_on_hwy=(gdf.geometry.length / gdf.route_length_feet).round(3) * 100,
    ).drop(
        columns=[
            "geometry",
        ]
    )

    # Join back the dataframe above with the original transit route dataframes
    # so we can have the original transit route geographies.
    gdf2 = pd.merge(
        routes_gdf,
        gdf,
        on=[
            "n_trips",
            "schedule_gtfs_dataset_key",
            "route_id",
            "route_type",
            "shape_id",
            "route_name_used",
            "route_length_feet",
        ],
        how="left",
    )

    # Clean up
    gdf2.district = gdf2.district.fillna(0).astype(int)
    return gdf2

In [38]:
intersecting = routes_shn_intersection(routes_assembled, 50)

In [39]:
len(intersecting)

20476

In [40]:
intersecting.pct_route_on_hwy.describe()

count   18951.00
mean        6.65
std        15.80
min         0.00
25%         0.10
50%         0.40
75%         2.70
max        99.20
Name: pct_route_on_hwy, dtype: float64

In [41]:
intersecting.columns

Index(['n_trips', 'geometry', 'schedule_gtfs_dataset_key', 'route_id',
       'route_type', 'shape_id', 'route_name_used', 'route_length_feet',
       'shn_route', 'district', 'highway_feet', 'pct_route_on_hwy'],
      dtype='object')

### Find multi route districts

In [42]:
# Find routes that cross multiple districts
multi_district_routes = (
    intersecting.groupby(["schedule_gtfs_dataset_key", "route_name_used", "route_id"])
    .agg({"district": "nunique"})
    .reset_index()
)

In [43]:
multi_district_routes.district.describe()

count   2683.00
mean       1.09
std        0.38
min        1.00
25%        1.00
50%        1.00
75%        1.00
max        7.00
Name: district, dtype: float64

In [45]:
multi_district_routes.sort_values(by=["district"], ascending=False).head(10)

,schedule_gtfs_dataset_key,route_name_used,route_id,district
1620,a37760dde6b9fdcb76b82e57afab7274,Greyhound US0831,US0831,7
1513,a37760dde6b9fdcb76b82e57afab7274,FlixBus N2003,N2003,6
555,48e137bc977da88970393f629c18432c,Coast Starlight,36924,5
549,48e137bc977da88970393f629c18432c,California Zephyr,96,4
1615,a37760dde6b9fdcb76b82e57afab7274,Greyhound US0800,US0800,4
1616,a37760dde6b9fdcb76b82e57afab7274,Greyhound US0802,US0802,4
1617,a37760dde6b9fdcb76b82e57afab7274,Greyhound US0810,US0810,4
582,48e137bc977da88970393f629c18432c,Texas Eagle,87,4
1619,a37760dde6b9fdcb76b82e57afab7274,Greyhound US0830,US0830,4
1618,a37760dde6b9fdcb76b82e57afab7274,Greyhound US0811,US0811,4


## Routes that overlap with multiple SHN now have 1+ row. Change it so one route will only have one row.

In [ ]:
intersecting.columns

In [46]:
def group_route_district(df: pd.DataFrame, pct_route_on_hwy_agg: str) -> pd.DataFrame:
    """
    Aggregate by adding all the districts and SHN to a single row, rather than
    multiple and sum up the total % of SHN a transit route intersects with.

    df: the dataframe you want to aggregate
    pct_route_on_hwy_agg: whether you want to find the max, min, sum, etc on the column
    "pct_route_on_hwy_across_districts"
    """

    agg1 = (
        df.groupby(
            ["schedule_gtfs_dataset_key", "route_type", "shape_id", "route_id", "route_name_used"],
            as_index=False,
        )[["shn_route", "shn_districts", "pct_route_on_hwy_across_districts"]]
        .agg(
            {
                "shn_route": lambda x: ", ".join(set(x.astype(str))),
                "shn_districts": lambda x: ", ".join(set(x.astype(str))),
                "pct_route_on_hwy_across_districts": pct_route_on_hwy_agg,
            }
        )
        .reset_index(drop=True)
    )

    # Clean up
    agg1.pct_route_on_hwy_across_districts = (
        agg1.pct_route_on_hwy_across_districts.astype(float).round(2)
    )
    return agg1

In [61]:
def add_shn_information(gdf: gpd.GeoDataFrame, buffer_amt:int) -> pd.DataFrame:
    """
    Prepare the gdf to join with the existing transit_routes
    dataframe that is published on the Open Data Portal
    """
    # Overlay
    intersecting = routes_shn_intersection(gdf, buffer_amt)
    # Rename column
    gdf = gdf.rename(columns={"pct_route_on_hwy": "pct_route_on_hwy_across_districts",
                             "district":"shn_districts"})
    # Group the dataframe so that one route only has one
    # row instead of multiple rows after finding its
    # intersection with any SHN routes.
    agg1 = group_route_district(gdf, "sum")

    # Add yes/no column to signify if a transit route intersects
    # with a SHN route
    agg1["on_shs"] = np.where(agg1["pct_route_on_hwy_across_districts"] == 0, "N", "Y")

    # Clean up rows that are tagged as "on_shs==N" but still have values
    # that appear. 
    agg1.loc[(agg1['on_shs'] == "N") & (agg1['shn_districts'] != "0"), 
                        ['shn_districts', 'shn_route']] = "NA"
    return agg1

In [62]:
open_data_portal_df = add_shn_information(intersecting, SHN_HWY_BUFFER_FEET)

In [51]:
len(open_data_portal_df)

7416

In [52]:
open_data_portal_df.columns

Index(['schedule_gtfs_dataset_key', 'route_type', 'shape_id', 'route_id',
       'route_name_used', 'shn_route', 'shn_districts',
       'pct_route_on_hwy_across_districts', 'on_shs'],
      dtype='object')

In [53]:
open_data_portal_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7416 entries, 0 to 7415
Data columns (total 9 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   schedule_gtfs_dataset_key          7416 non-null   object 
 1   route_type                         7416 non-null   object 
 2   shape_id                           7416 non-null   object 
 3   route_id                           7416 non-null   object 
 4   route_name_used                    7416 non-null   object 
 5   shn_route                          7407 non-null   object 
 6   shn_districts                      7407 non-null   object 
 7   pct_route_on_hwy_across_districts  7416 non-null   float64
 8   on_shs                             7416 non-null   object 
dtypes: float64(1), object(8)
memory usage: 521.6+ KB


In [54]:
open_data_portal_df.pct_route_on_hwy_across_districts.describe()

count   7416.00
mean      16.98
std       27.33
min        0.00
25%        0.40
50%        1.40
75%       21.42
max      100.00
Name: pct_route_on_hwy_across_districts, dtype: float64

In [55]:
open_data_portal_df.on_shs.value_counts()

Y    5882
N    1534
Name: on_shs, dtype: int64

In [56]:
open_data_portal_df.columns

Index(['schedule_gtfs_dataset_key', 'route_type', 'shape_id', 'route_id',
       'route_name_used', 'shn_route', 'shn_districts',
       'pct_route_on_hwy_across_districts', 'on_shs'],
      dtype='object')

## Missing Routes for D2 [PR](https://github.com/cal-itp/data-analyses/issues/1582)

In [63]:
open_data_portal_df.loc[open_data_portal_df.shn_districts.str.contains("2")]

,schedule_gtfs_dataset_key,route_type,shape_id,route_id,route_name_used,shn_route,shn_districts,pct_route_on_hwy_across_districts,on_shs
620,0666caf3ec1ecc96b74f4477ee4bc939,3,4600199_DEC24,460-13186,460,"10, 105, 91, 110, 605, 39, 5, 710","12, 7",48.50,Y
621,0666caf3ec1ecc96b74f4477ee4bc939,3,4600200_DEC24,460-13186,460,"10, 105, 91, 110, 605, 39, 5, 710","12, 7",47.90,Y
622,0666caf3ec1ecc96b74f4477ee4bc939,3,4600205_DEC24,460-13186,460,"10, 105, 91, 110, 605, 39, 5, 710","12, 7",50.00,Y
623,0666caf3ec1ecc96b74f4477ee4bc939,3,4600206_DEC24,460-13186,460,"10, 105, 91, 110, 605, 39, 5, 710","12, 7",50.20,Y
666,0666caf3ec1ecc96b74f4477ee4bc939,3,5770040_DEC24,577-13186,577,"10, 105, 91, 72, 60, 22, 605, 5, 405","12, 7",88.10,Y
667,0666caf3ec1ecc96b74f4477ee4bc939,3,5770042_DEC24,577-13186,577,"10, 105, 91, 72, 1, 60, 22, 605, 5, 405","12, 7",86.20,Y
1015,0d65d96d07115e28313f207d5ed0d3b2,3,p_110891,6332,Reno,"395, 70, 36","2, 3",85.80,Y
1016,0d65d96d07115e28313f207d5ed0d3b2,3,p_110892,6332,Reno,"395, 36","2, 3",85.30,Y
1466,330fd5b796496eddf2ce1d1b9828961d,3,p_1432980,32882,1 - Cascade Flyer (Express),"97, 265, 5, 3, 89",2,87.00,Y
1467,330fd5b796496eddf2ce1d1b9828961d,3,p_1432981,32882,1 - Cascade Flyer (Express),"97, 265, 5, 3, 89",2,89.20,Y


### Map

In [59]:
m = shn_district_df.explore(
    name="district",
    tiles="CartoDB positron",
    style_kwds={"color": "#9DA4A6", "opacity": 0.5},
    height=500,
    width=1000,
    legend = False
)

NameError: name 'shn_district_df' is not defined

In [ ]:
southwest_chief = intersecting.loc[ (intersecting.route_name_used ==  "Southwest Chief") 
]

In [ ]:
"""southwest_chief.explore(
    m=m,
    cmap="Spectral",
    categorical=True,
    legend=False,
    legend_kwds={"width": 200},
)"""